In [26]:
from bs4 import BeautifulSoup
import requests

In [27]:
# URL = "https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=Nuggets&BeginDate=2022-01-01&EndDate=2022-04-06&PlayerMovementChkBx=yes&ILChkBx=yes&Submit=Search"
# page = requests.get(URL)

In [28]:
# soup = BeautifulSoup(page.content, "html.parser")
# table = soup.find("table", class_="datatable")
# headers = [header.text[1:] for header in table.find('tr',class_="DraftTableLabel").findAll('td')]
# results = [{headers[i]: cell.text for i, cell in enumerate(row.findAll('td'))}
#            for row in table.findAll('tr')[1:]]
# print(headers)
# print(results)

In [29]:
# for i, item in enumerate(results):
#     print(item['Acquired'][3:])

In [30]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import playergamelogs
import time

In [31]:
sleeptime = .7
headers = ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID']
rosters = {}
player_logs= {}
for team in teams.get_teams():
    regular_season = playergamelogs.PlayerGameLogs(season_nullable="2021-22", team_id_nullable=team["id"], season_type_nullable="Regular Season")
    time.sleep(sleeptime)
    playins = playergamelogs.PlayerGameLogs(season_nullable="2021-22", team_id_nullable=team["id"], season_type_nullable="PlayIn")
    time.sleep(sleeptime)
    playoffs = playergamelogs.PlayerGameLogs(season_nullable="2021-22", team_id_nullable=team["id"], season_type_nullable="Playoffs")
    time.sleep(sleeptime)
    team_player_logs = regular_season.get_dict()["resultSets"][0]['rowSet'] + playins.get_dict()["resultSets"][0]['rowSet'] + playoffs.get_dict()["resultSets"][0]['rowSet']
    print(team["id"], len(team_player_logs))
    player_logs[team['city'] + " " + team['nickname']] = [(player[2], player[8][:10], player[6]) for player in team_player_logs]
    roster = [(player[3],player[14]) for player in commonteamroster.CommonTeamRoster(team["id"], season="2021-22").common_team_roster.get_dict()['data']]
    time.sleep(sleeptime)
    rosters[team['city'] + " " + team['nickname']]= roster
print(rosters)

1610612737 960
1610612738 1012
1610612739 889
1610612740 968
1610612741 891
1610612742 1084
1610612743 959
1610612744 1057
1610612745 867
1610612746 882
1610612747 828
1610612748 977
1610612749 1020
1610612750 998
1610612751 909
1610612752 840
1610612753 840
1610612754 838
1610612755 974
1610612756 1007
1610612757 846
1610612758 834


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
rosters["LA Clippers"] = rosters["Los Angeles Clippers"]
player_logs["LA Clippers"] = player_logs["Los Angeles Clippers"]

In [ ]:
URL_ESPN = "https://www.espn.com/nba/injuries"
espn = requests.get(URL_ESPN)

In [ ]:
soup = BeautifulSoup(espn.content, "html.parser")
tables = [table for table in soup.find_all("div", {"class": "Table__league-injuries"})]
teams_injured = [{"team":table.find("span", {"class":"injuries__teamName"}).text} for table in tables]

for i, table  in enumerate(tables):
    team = teams_injured[i]
    teams_injured[i]["injury_list"] = [{"name":item.find("a").text, "date":item.find("td",{"class":"col-date"}).text, "status":item.find("td",{"class":"col-stat"}).text} for item in table.find_all("tr", {"data-idx":True})]

In [ ]:
from datetime import datetime
return_date = datetime.now().strftime("%Y-%m-%d")
print(return_date)

2022-05-18


In [ ]:
url = ""
with open("../app.env") as file:
    db_source = file.read()
    url = db_source.split("DB_SOURCE=\"")[1][:-1]

In [ ]:
from pymongo import MongoClient
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import time
client = MongoClient(url)
db = client.nba

In [ ]:

# for each team on the injury report
for team in teams_injured:
    # get the team's game logs
    team_logs = [log for log in player_logs[team["team"]] if log[2] == team["team"]]
    # and sort them by recent date
    team_logs.sort(key=lambda date: datetime.strptime(date[1], "%Y-%m-%d"), reverse=True)
    # get the last game the team played
    last_team_game_played = team_logs[0][1]
    print(team["team"], last_team_game_played)
    # for each player currently on the team's roster
    for player in rosters[team["team"]]:
        # get the list of games the player played for this team
        played_with_team = [log[1] for log in team_logs if log[0] == player[0]]
        # and sort it by recent date
        played_with_team.sort(key=lambda date: datetime.strptime(date, "%Y-%m-%d"), reverse=True)
        # if the player has played a game for this team
        if len(played_with_team) > 0:
            # then get the last game (date) played by the player
            last_game_played_with_team = played_with_team[0]
        else:
            last_game_played_with_team = ""

        # if the player name is found on the injury report
        if player[0] in [player["name"] for player in team["injury_list"]]:
            # if they played in the last game
            if last_game_played_with_team == last_team_game_played:
                # then the injury must be new
                # push new injury event object with start date as today (note: handle OUT vs day-to-day)
                p=[p for p in team["injury_list"] if player[0] == p["name"]][0]
                print(player, " Start Injury ", "Today", p)
                injury_data = {
                    "start_date":last_game_played_with_team,
                    "return_date":"",
                    "playername":player[0],
                    "playerID":player[1],
                    "status":p["status"],
                    "lastModifiedDate":return_date
                }
                print(injury_data)
                db.injuries.replace_one({"playerID":player[1],"return_date":""}, injury_data, upsert=True)
            else:
                #continued injury, (update games missed on object?) -- do nothing for now
                p=[p for p in team["injury_list"] if player[0] == p["name"]][0]
                print(player, " Injured since ", last_game_played_with_team)
                db.injuries.update_one({"playerID":player[1],"return_date":""}, {"$set":{"status":p["status"], "start_date":last_game_played_with_team, "lastModifiedDate":return_date}}, upsert=True)
        # healthy
        else:
            # determine if a player has returned from injury
            if last_game_played_with_team != last_team_game_played:
                #update injury event object with end date
                print(player, " Returning from injury since", last_game_played_with_team)
                # TODO: Right now only updating the return date of injured players that have previously been added to the database through the injury report scraper
                # TODO: Need to get a history of injuries from another source
                db.injuries.update_one({"playerID":player[1],"return_date":""}, {"$set":{"return_date":return_date, "start_date":last_game_played_with_team, "lastModifiedDate":return_date}}, upsert=False)
            
    print("-----")

Atlanta Hawks 2022-04-26
('Jalen Johnson', 1630552)  Returning from injury since 2022-04-24
('Sharife Cooper', 1630536)  Returning from injury since 2022-04-10
('Skylar Mays', 1630219)  Returning from injury since 2022-04-24
('Kevin Knox II', 1628995)  Returning from injury since 2022-04-24
('Lou Williams', 101150)  Injured since  2022-04-05
('Gorgui Dieng', 203476)  Returning from injury since 2022-04-24
('Bogdan Bogdanovic', 203992)  Injured since  2022-04-24
('Clint Capela', 203991)  Start Injury  Today {'name': 'Clint Capela', 'date': 'Apr 26', 'status': 'Day-To-Day'}
{'start_date': '2022-04-26', 'return_date': '', 'playername': 'Clint Capela', 'playerID': 203991, 'status': 'Day-To-Day', 'lastModifiedDate': '2022-05-18'}
('Chaundee Brown Jr.', 1630602)  Returning from injury since 2021-12-31
-----
Boston Celtics 2022-05-17
('Matt Ryan', 1630346)  Returning from injury since 2022-04-10
('Malik Fitts', 1630238)  Returning from injury since 2022-05-15
('Nik Stauskas', 203917)  Returni